<a href="https://colab.research.google.com/github/alexacheui/ateema_capstone/blob/main/video_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/MyDrive/ADSP Capstone')

In [4]:
!pwd

/content/drive/MyDrive/ADSP Capstone


In [20]:
!pip install moviepy opencv-python

In [25]:
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip,concatenate_videoclips
import cv2
import numpy as np

# Define a function to create a mask for the video frame
def create_mask(frame, lower_bound, upper_bound):
    # Convert frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Create a mask with the given color bounds
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    # Invert the mask to get the foreground
    mask_inv = cv2.bitwise_not(mask)
    return mask_inv

# Define the function to process each frame
def process_frame(frame):
    # Define color bounds (example: removing green background)
    lower_bound = np.array([35, 100, 100])  # Lower bound of green in HSV
    upper_bound = np.array([85, 255, 255])  # Upper bound of green in HSV
    # Create mask
    mask = create_mask(frame, lower_bound, upper_bound)
    # Convert mask to 3 channels
    mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    # Apply mask to the frame
    fg = cv2.bitwise_and(frame, mask_3ch)
    return fg

# Load and trim the base video to 1 minute
video = VideoFileClip("avatar.mp4").subclip(0, 30)  # Trim to the first 60 seconds

# Apply the process_frame function to each frame in the video
processed_video = video.fl_image(process_frame)

# Load the background images and set their duration to 30 seconds each
background1 = ImageClip("background.jpeg").set_duration(10).resize(width=320)
background2 = ImageClip("background2.jpeg").set_duration(20).resize(width=320)

# Concatenate the two background images to match the 1-minute video duration
background = concatenate_videoclips([background1, background2])

# Resize the video to a width of 100 pixels and set its position slightly away from the left edge
processed_video = processed_video.resize(width=100).set_pos((10, "center"))

# Create a composite clip with the background and the resized video
final_video = CompositeVideoClip([background, processed_video])

# Write the final video to a file
final_video.write_videofile("test.mp4", codec="libx264")


Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4


In [26]:
from IPython.display import HTML
from base64 import b64encode

# Function to display video
def display_video(filename):
    video_path = f'/content/drive/MyDrive/ADSP Capstone/{filename}'
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'''
    <video width=600 controls>
          <source src="{data_url}" type="video/mp4">
    </video>
    ''')

# Display the video
display_video("test.mp4")